### Import Lybrary

In [42]:
import pandas as pd
import numpy as np
import re

In [ ]:
file_list = [
r'C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_2020_2021.csv',
r'C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv',
r'C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv',
r'C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2024.csv' 
]

df_national_examination_board = pd.read_excel(
r"C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\danh_sach_hoi_dong_thi.xlsx"
)

class NationalHighSchoolExamScore:
    def __init__(self, file_paths, df_national_examination_board):
        self.file_paths = file_paths
        self.dataframes = []
        self.df_national_examination_board = df_national_examination_board

    def read_data(self):
        for path in self.file_paths:
            try:
                df = pd.read_csv(path)
                self.dataframes.append((path, df))
                print(f"Read file: {path}")
            except Exception as e:
                print(f"Error reading file {path}: {e}")

    def check_data(self):
            if not self.dataframes:
                print("No data available")
                return

            for path, df in self.dataframes:
                print(f"\n Check data: {path}")
                print(f"Row numbers: {df.shape[0]}, Column numbers: {df.shape[1]}")
                print("Columns:", list(df.columns))
                print("Duplicate Values:", df.duplicated().sum())
                print("-" * 60)

    def remove_duplicate(self):
        for i, (path, df) in enumerate(self.dataframes):
            before = len(df)
            df = df.drop_duplicates()
            self.dataframes[i] = (path, df)
            print(f"Removed {before - len(df)} duplicate rows from: {path}")

    def drop_specific_columns(self, drop_map):
        for i, (path, df) in enumerate(self.dataframes):
            for key, columns in drop_map.items():
                if key in path:
                    df = df.drop(columns=[col for col in columns if col in df.columns], errors='ignore')
                    self.dataframes[i] = (path, df)

    def rename_columns(self, rename_map):
        for i, (path, df) in enumerate(self.dataframes):
            for key, renames in rename_map.items():
                if key in path:
                    df = df.rename(columns={k: v for k, v in renames.items() if k in df.columns})
                    self.dataframes[i] = (path, df)

    def add_column_code_year(self):
        for i, (path, df) in enumerate(self.dataframes):
            if 'code' not in df.columns and 'sbd' in df.columns:
                df['code'] = df['sbd'].astype(str).str[:2].astype(int)
                print(f" Added 'code' to file: {path}")
            if 'year' not in df.columns:
                df['year'] = 2022 + i - 1  
                print(f"Added 'year' = {2022 + i - 1} to file: {path}")
            self.dataframes[i] = (path, df)
    
    def check_data_column_year(self):
        if not self.dataframes:
            print("No data to process")
            return
        for path, df in self.dataframes:
            print(df["year"].unique())
            

    def reorder_all_columns(self):
        desired_order = [
            'sbd', 'toan', 'ngu_van', 'vat_li', 'hoa_hoc', 'sinh_hoc',
            'lich_su', 'dia_li', 'gdcd', 'ngoai_ngu', 'code', 'year'
        ]
        for i, (path, df) in enumerate(self.dataframes):
            ordered_cols = [col for col in desired_order if col in df.columns]
            df = df[ordered_cols + [col for col in df.columns if col not in ordered_cols]]
            self.dataframes[i] = (path, df)

    def concat_all(self):
        all_dfs = [df for _, df in self.dataframes]
        df_concat = pd.concat(all_dfs, ignore_index=True)
        print(f"📦 Total merged rows: {len(df_concat)}")
        return df_concat

    def add_column_khtn_khxh_khoia_khoib_khoic_khoid(self, df):
        df['khtn'] = df[['vat_li', 'hoa_hoc', 'sinh_hoc']].sum(axis=1, skipna=False)
        df['khtn'] = df['khtn'].where(df[['vat_li', 'hoa_hoc', 'sinh_hoc']].notnull().all(axis=1))

        df['khxh'] = df[['lich_su', 'dia_li', 'gdcd']].sum(axis=1, skipna=False)
        df['khxh'] = df['khxh'].where(df[['lich_su', 'dia_li', 'gdcd']].notnull().all(axis=1))

        df['khoi_a'] = df[['toan', 'vat_li', 'hoa_hoc']].sum(axis=1, skipna=False)
        df['khoi_a'] = df['khoi_a'].where(df[['toan', 'vat_li', 'hoa_hoc']].notnull().all(axis=1))

        df['khoi_b'] = df[['toan', 'hoa_hoc', 'sinh_hoc']].sum(axis=1, skipna=False)
        df['khoi_b'] = df['khoi_b'].where(df[['toan', 'hoa_hoc', 'sinh_hoc']].notnull().all(axis=1))

        df['khoi_c'] = df[['ngu_van', 'lich_su', 'dia_li']].sum(axis=1, skipna=False)
        df['khoi_c'] = df['khoi_c'].where(df[['ngu_van', 'lich_su', 'dia_li']].notnull().all(axis=1))

        df['khoi_d'] = df[['toan', 'ngu_van', 'ngoai_ngu']].sum(axis=1, skipna=False)
        df['khoi_d'] = df['khoi_d'].where(df[['toan', 'ngu_van', 'ngoai_ngu']].notnull().all(axis=1))

        print(" Success add columns: khtn, khxh, group A-D")
        return df

In [44]:
data = NationalHighSchoolExamScore(file_list,df_national_examination_board)

#### Read Data CSV

In [45]:
data.read_data()

C:\Users\Phi Huynh\AppData\Local\Temp\ipykernel_29892\3500005507.py:21: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Read file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_2020_2021.csv
Read file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
Read file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
Read file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2024.csv


#### Check Information Data

In [46]:
data.check_data()


 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_2020_2021.csv
Row numbers: 1857877, Column numbers: 16
Columns: ['SBD', 'Tên', 'Ngày Sinh', 'Giới tính', 'Toán', 'Văn', 'Lý', 'Hoá', 'Sinh', 'Lịch Sử', 'Địa Lý', 'GDCD', 'Ngoại Ngữ', 'Year', 'code', 'province']
Duplicate Values: 4591
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
Row numbers: 995441, Column numbers: 10
Columns: ['sbd', 'toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
Row numbers: 1022060, Column numbers: 11
Columns: ['sbd', 'toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd', 'ma_ngoai_ngu']
Duplicate Values: 0
--------

#### Remove Lines Duplicate

In [47]:
data.remove_duplicate()

Removed 4591 duplicate rows from: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_2020_2021.csv
Removed 0 duplicate rows from: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
Removed 0 duplicate rows from: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
Removed 0 duplicate rows from: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2024.csv


#### Check Information Data

In [48]:
data.check_data()


 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_2020_2021.csv
Row numbers: 1853286, Column numbers: 16
Columns: ['SBD', 'Tên', 'Ngày Sinh', 'Giới tính', 'Toán', 'Văn', 'Lý', 'Hoá', 'Sinh', 'Lịch Sử', 'Địa Lý', 'GDCD', 'Ngoại Ngữ', 'Year', 'code', 'province']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
Row numbers: 995441, Column numbers: 10
Columns: ['sbd', 'toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
Row numbers: 1022060, Column numbers: 11
Columns: ['sbd', 'toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd', 'ma_ngoai_ngu']
Duplicate Values: 0
-----------

#### Drop Columns Specific

In [49]:
columns_to_drop = {
    "2020_2021": ["Tên", "Ngày Sinh", "Giới tính"],
    "2023": ["ma_ngoai_ngu"],
    "2024": ["ma_ngoai_ngu"]
}
data.drop_specific_columns(columns_to_drop)

#### Check Information Data

In [50]:
data.check_data()


 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_2020_2021.csv
Row numbers: 1853286, Column numbers: 13
Columns: ['SBD', 'Toán', 'Văn', 'Lý', 'Hoá', 'Sinh', 'Lịch Sử', 'Địa Lý', 'GDCD', 'Ngoại Ngữ', 'Year', 'code', 'province']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
Row numbers: 995441, Column numbers: 10
Columns: ['sbd', 'toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
Row numbers: 1022060, Column numbers: 10
Columns: ['sbd', 'toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd']
Duplicate Values: 0
------------------------------------------------------------

#### Rename Column

In [51]:
data.rename_columns({
    "2020_2021": {"SBD": "sbd", "Toán": "toan", "Văn": "ngu_van", "Ngoại Ngữ": "ngoai_ngu", "Lý": "vat_li", "Hoá": "hoa_hoc", 
                  "Sinh": "sinh_hoc", "Lịch Sử": "lich_su", "Địa Lý": "dia_li", "GDCD": "gdcd", "Year": "year"}
})


#### Check Information Data

In [52]:
data.check_data()


 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_2020_2021.csv
Row numbers: 1853286, Column numbers: 13
Columns: ['sbd', 'toan', 'ngu_van', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd', 'ngoai_ngu', 'year', 'code', 'province']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
Row numbers: 995441, Column numbers: 10
Columns: ['sbd', 'toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
Row numbers: 1022060, Column numbers: 10
Columns: ['sbd', 'toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd']
Duplicate Values: 0
--------------------------------------------

#### Add Columns Code And Year

In [53]:
data.add_column_code_year()

 Added 'code' to file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
Added 'year' = 2022 to file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
 Added 'code' to file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
Added 'year' = 2023 to file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
 Added 'code' to file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2024.csv
Added 'year' = 2024 to file: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2024.csv


#### Check Information Data

In [54]:
data.check_data_column_year()

[2020 2021]
[2022]
[2023]
[2024]


#### Reorder The Columns

In [55]:
data.reorder_all_columns()

#### Check Information Data

In [56]:
data.check_data()


 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_2020_2021.csv
Row numbers: 1853286, Column numbers: 13
Columns: ['sbd', 'toan', 'ngu_van', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd', 'ngoai_ngu', 'code', 'year', 'province']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2022.csv
Row numbers: 995441, Column numbers: 12
Columns: ['sbd', 'toan', 'ngu_van', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd', 'ngoai_ngu', 'code', 'year']
Duplicate Values: 0
------------------------------------------------------------

 Check data: C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\diem_thi_thpt_2023.csv
Row numbers: 1022060, Column numbers: 12
Columns: ['sbd', 'toan', 'ngu_van', 'vat_li', 'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd', 'ngoai_ngu', 'code', 'year']
Duplicate Values: 0
------------

#### Concat Table

In [57]:
df_national_high_school_exam_score = data.concat_all()

📦 Total merged rows: 4932392


#### Check Information Data

In [58]:
df_national_high_school_exam_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932392 entries, 0 to 4932391
Data columns (total 13 columns):
 #   Column     Dtype  
---  ------     -----  
 0   sbd        int64  
 1   toan       float64
 2   ngu_van    float64
 3   vat_li     float64
 4   hoa_hoc    float64
 5   sinh_hoc   float64
 6   lich_su    float64
 7   dia_li     float64
 8   gdcd       float64
 9   ngoai_ngu  float64
 10  code       int64  
 11  year       int64  
 12  province   object 
dtypes: float64(9), int64(3), object(1)
memory usage: 489.2+ MB


In [59]:
df_national_high_school_exam_score = data.add_column_khtn_khxh_khoia_khoib_khoic_khoid(df_national_high_school_exam_score)
df_national_high_school_exam_score.to_csv(r"C:\FPT Polytechnic\Project Tự Làm\Điểm thi thpt 2020 - 2024\processed.csv",index=False,encoding='utf-8-sig')

 Success add columns: khtn, khxh, khối A-D


#### Rename column danh_sach_hoi_dong_thi.xlsx

In [ ]:
df_national_examination_board.rename(columns={
"Mã hội đồng": "code",
"Tên hội đồng thi": "national examination board",
"Tên Tỉnh": "province"
}, inplace=True)

In [ ]:
df_national_examination_board.to_csv(r"C:\FPT Polytechnic\Graduation_Project\Data\Processed\national_examination_board.csv", index=False)
df_region_of_vietnam = pd.read_excel(r"C:\FPT Polytechnic\Graduation_Project\Data\Raw\Regions_of_VietNam.xlsx")
df_national_examination_board_transform = pd.merge(df_national_examination_board, df_region_of_vietnam, how="left", left_on='province', right_on='Province')
df_national_examination_board_transform.info()
df_national_examination_board_transform.drop(columns='Province', inplace=True)
df_national_examination_board_transform.rename(columns={'Regions': 'regions'}, inplace=True)
df_national_examination_board_transform.to_csv(r'C:\FPT Polytechnic\Graduation_Project\Data\Processed\national_examination_board_transform.csv', index=False)
df_national_high_school_exam_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   code                        64 non-null     int64 
 1   national examination board  64 non-null     object
 2   province                    64 non-null     object
 3   Province                    63 non-null     object
 4   Regions                     63 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932392 entries, 0 to 4932391
Data columns (total 19 columns):
 #   Column     Dtype  
---  ------     -----  
 0   sbd        int64  
 1   toan       float64
 2   ngu_van    float64
 3   vat_li     float64
 4   hoa_hoc    float64
 5   sinh_hoc   float64
 6   lich_su    float64
 7   dia_li     float64
 8   gdcd       float64
 9   ngoai_ngu  float64
 10  code       int64  
 11  year       int64  
 12  pr

In [61]:
df_national_high_school_exam_score

,sbd,toan,ngu_van,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd,ngoai_ngu,code,year,province,khtn,khxh,khoi_a,khoi_b,khoi_c,khoi_d
0,18014547,6.4,6.75,NaN,NaN,NaN,4.75,7.00,6.50,4.2,18,2020,Bắc Giang,NaN,18.25,NaN,NaN,18.50,17.35
1,18014530,7.6,6.00,NaN,NaN,NaN,3.75,7.75,7.75,2.8,18,2020,Bắc Giang,NaN,19.25,NaN,NaN,17.50,16.40
2,18014521,4.8,4.75,NaN,NaN,NaN,4.00,6.50,NaN,NaN,18,2020,Bắc Giang,NaN,NaN,NaN,NaN,15.25,NaN
3,18014517,8.0,7.00,NaN,NaN,NaN,8.25,8.00,9.50,5.8,18,2020,Bắc Giang,NaN,25.75,NaN,NaN,23.25,20.80
4,18014523,8.2,6.50,8.00,8.5,5.0,NaN,NaN,NaN,4.0,18,2020,Bắc Giang,21.50,NaN,24.70,21.7,NaN,18.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4932387,64007282,4.2,6.00,NaN,NaN,NaN,5.25,6.25,7.50,3.0,64,2024,NaN,NaN,19.00,NaN,NaN,17.50,13.20
4932388,64007283,5.2,6.00,NaN,NaN,NaN,6.25,5.75,7.75,3.6,64,2024,NaN,NaN,19.75,NaN,NaN,18.00,14.80
4932389,64007284,7.2,8.00,6.25,7.0,8.0,NaN,NaN,NaN,7.0,64,2024,NaN,21.25,NaN,20.45,22.2,NaN,22.20
4932390,64007285,5.2,5.25,NaN,NaN,NaN,4.75,6.50,8.75,4.4,64,2024,NaN,NaN,20.00,NaN,NaN,16.50,14.85


In [62]:
df_national_high_school_exam_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932392 entries, 0 to 4932391
Data columns (total 19 columns):
 #   Column     Dtype  
---  ------     -----  
 0   sbd        int64  
 1   toan       float64
 2   ngu_van    float64
 3   vat_li     float64
 4   hoa_hoc    float64
 5   sinh_hoc   float64
 6   lich_su    float64
 7   dia_li     float64
 8   gdcd       float64
 9   ngoai_ngu  float64
 10  code       int64  
 11  year       int64  
 12  province   object 
 13  khtn       float64
 14  khxh       float64
 15  khoi_a     float64
 16  khoi_b     float64
 17  khoi_c     float64
 18  khoi_d     float64
dtypes: float64(15), int64(3), object(1)
memory usage: 715.0+ MB
